In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict


In [5]:
# Primeiro, vamos contar a quantidade total de registros
dataset = pd.read_csv(r'C:/Users/Pedro Henrique/PGC/input/Tweets_Mg.csv',encoding='utf-8')
dataset.count()

Unnamed: 0                   8199
Created At                   8199
Text                         8199
Geo Coordinates.latitude      104
Geo Coordinates.longitude     104
User Location                5489
Username                     8199
User Screen Name             8199
Retweet Count                8199
Classificacao                8199
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [6]:
# Agora, apenas os classificados como neutro
dataset[dataset.Classificacao == 'Neutro'].count()

Unnamed: 0                   2453
Created At                   2453
Text                         2453
Geo Coordinates.latitude      102
Geo Coordinates.longitude     102
User Location                1712
Username                     2453
User Screen Name             2453
Retweet Count                2453
Classificacao                2453
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [7]:
# Os classificados como positivo
dataset[dataset.Classificacao == 'Positivo'].count()



Unnamed: 0                   3300
Created At                   3300
Text                         3300
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                2118
Username                     3300
User Screen Name             3300
Retweet Count                3300
Classificacao                3300
Observação                      1
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [8]:
# E finalmente, os classificados como negativo
dataset[dataset.Classificacao == 'Negativo'].count()

Unnamed: 0                   2446
Created At                   2446
Text                         2446
Geo Coordinates.latitude        1
Geo Coordinates.longitude       1
User Location                1659
Username                     2446
User Screen Name             2446
Retweet Count                2446
Classificacao                2446
Observação                      0
Unnamed: 10                     0
Unnamed: 11                     0
Unnamed: 12                     0
Unnamed: 13                     0
Unnamed: 14                     0
Unnamed: 15                     0
Unnamed: 16                     0
Unnamed: 17                     0
Unnamed: 18                     0
Unnamed: 19                     0
Unnamed: 20                     0
Unnamed: 21                     0
Unnamed: 22                     0
Unnamed: 23                     0
Unnamed: 24                     0
dtype: int64

In [9]:
dataset.head()

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Próximo passo, vamos separar os tweets e suas classes
tweets = dataset["Text"].values
tweets



array(['���⛪ @ Catedral de Santo Antônio - Governador Valadares/MG https://t.co/JSbKamIqUJ',
       '� @ Governador Valadares, Minas Gerais https://t.co/B3ThIDJCSf',
       '�� @ Governador Valadares, Minas Gerais https://t.co/dPkgzVR2Qw',
       ...,
       'Trio é preso suspeito de roubo, tráfico e abuso sexual em Uberlândia https://t.co/zaQbXRRJWc',
       'Trio é preso suspeito de roubo, tráfico e abuso sexual em Uberlândia: Um dos autores teria molestado vítima de… https://t.co/lQ8cTSNftA',
       'Trio suspeito de roubo de cargas é preso em Santa Luzia (MG) https://t.co/0INgJcMtZb #R7MG #RecordTVMinas'],
      dtype=object)

In [11]:
classes = dataset["Classificacao"].values
classes

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype=object)

In [12]:
# Agora, vamos treinar o modelo usando a abordagem Bag of Words e o algoritmo Naive Bayes Multinomial
#    - Bag of Words, na prática, cria um vetor com cada uma das palavras do texto completo da base,
#      depois, calcula a frequência em que essas palavras ocorrem em uma data sentença, para então
#      classificar/treinar o modelo
#    - Exemplo HIPOTÉTICO de três sentenças vetorizadas "por palavra" e classificadas baseada na
#      frequência de suas palavras:
#         {0,3,2,0,0,1,0,0,0,1, Positivo}
#         {0,0,1,0,0,1,0,1,0,0, Negativo}
#         {0,1,1,0,0,1,0,0,0,0, Neutro}
#    - Olhando para esses vetores, meu palpite é que as palavras nas posições 2 e 3 são as com maior
#      peso na determinação de a que classe pertence cada uma das três sentenças avaliadas
#    - A função fit_transform faz exatamente esse processo: ajusta o modelo, aprende o vocabulário,
#      e transforma os dados de treinamento em feature vectors, a.k.a. vetor com frequêcia das palavras

vectorizer = CountVectorizer(analyzer = "word")
freq_tweets = vectorizer.fit_transform(tweets)

modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB()

In [13]:
# Vamos usar algumas frases de teste para fazer a classificação com o modelo treinado
testes = ["Esse governo está no início, vamos ver o que vai dar",
          "Estou muito feliz com o governo de São Paulo esse ano",
          "O estado de Minas Gerais decretou calamidade financeira!!!",
          "A segurança desse país está deixando a desejar",
          "O governador de Minas é do PT",
          "O prefeito de São Paulo está fazendo um ótimo trabalho"]

freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array(['Neutro', 'Neutro', 'Negativo', 'Negativo', 'Neutro', 'Positivo'],
      dtype='<U8')

In [14]:
# Validação cruzada do modelo. Neste caso, o modelo é dividido em 10 partes, treinado em 9 e testado em 1
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)
resultados

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype='<U8')

In [15]:
# Quão acurada é a média do modelo?
metrics.accuracy_score(classes, resultados)

0.8831564824978656

In [16]:
# Medidas de validação do modelo
sentimentos = ["Positivo", "Negativo", "Neutro"]
print(metrics.classification_report(classes, resultados, target_names = sentimentos))

# Lembrando que:
#    : precision = true positive / (true positive + false positive)
#    : recall    = true positive / (true positive + false negative)
#    : f1-score  = 2 * ((precision * recall) / (precision + recall))

              precision    recall  f1-score   support

    Positivo       0.89      0.93      0.91      2446
    Negativo       0.80      0.84      0.82      2453
      Neutro       0.95      0.88      0.91      3300

    accuracy                           0.88      8199
   macro avg       0.88      0.88      0.88      8199
weighted avg       0.89      0.88      0.88      8199



Melhorando o modelo

In [17]:
# Com o modelo de Bigrams, em lugar de vetorizar o texto "por palavra", vamos vetoriza-lo por cada
# "duas palavras", tipo: Eu gosto de São Paulo => { eu gosto, gosto de, de são, são paulo }
vectorizer = CountVectorizer(ngram_range = (1, 2))
freq_tweets = vectorizer.fit_transform(tweets)

modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB()

In [18]:
# Nova predição bigramada
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)
resultados

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype='<U8')

In [19]:
# Qual foi a acuracidade desse novo modelo?
metrics.accuracy_score(classes, resultados)

0.8954750579338944

In [20]:
# As novas medidas de validação do modelo, um pouquinho melhor que o anterior
print(metrics.classification_report(classes, resultados, target_names = sentimentos))

              precision    recall  f1-score   support

    Positivo       0.91      0.93      0.92      2446
    Negativo       0.80      0.89      0.84      2453
      Neutro       0.97      0.88      0.92      3300

    accuracy                           0.90      8199
   macro avg       0.89      0.90      0.89      8199
weighted avg       0.90      0.90      0.90      8199



Gaussian Naive Bayes

In [30]:
from sklearn.naive_bayes import GaussianNB

vectorizer = CountVectorizer(ngram_range = (1, 2))
freq_tweets = vectorizer.fit_transform(tweets)


gaussian_nb_model = GaussianNB()
gaussian_nb_model.fit(freq_tweets.toarray(), classes)

GaussianNB()

In [32]:
resultados = cross_val_predict(gaussian_nb_model, freq_tweets.toarray(), classes, cv = 10)
resultados

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype='<U8')

In [33]:
metrics.accuracy_score(classes, resultados)

0.8633979753628491

In [34]:
print(metrics.classification_report(classes, resultados, target_names = sentimentos))

              precision    recall  f1-score   support

    Positivo       0.90      0.88      0.89      2446
    Negativo       0.75      0.85      0.80      2453
      Neutro       0.94      0.86      0.90      3300

    accuracy                           0.86      8199
   macro avg       0.86      0.86      0.86      8199
weighted avg       0.87      0.86      0.87      8199



SVM

In [ ]:
from sklearn import svm

vectorizer = CountVectorizer(ngram_range = (1, 2))
freq_tweets = vectorizer.fit_transform(tweets)

svm_model = svm.SVC(kernel='linear')

svm_model.fit(freq_tweets, classes)

SVC(kernel='linear')

In [45]:
resultados = cross_val_predict(svm_model, freq_tweets, classes, cv = 10)
resultados

array(['Neutro', 'Neutro', 'Neutro', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype=object)

In [46]:
metrics.accuracy_score(classes, resultados)

0.8875472618612026

In [47]:
print(metrics.classification_report(classes, resultados, target_names = sentimentos))

              precision    recall  f1-score   support

    Positivo       0.93      0.89      0.91      2446
    Negativo       0.77      0.91      0.83      2453
      Neutro       0.97      0.87      0.92      3300

    accuracy                           0.89      8199
   macro avg       0.89      0.89      0.89      8199
weighted avg       0.90      0.89      0.89      8199



Análise Binária (Positivo e Negativo)

In [57]:
df_binario = dataset[dataset.Classificacao != 'Neutro']

tweets_binario = df_binario["Text"].values

classes_binario = df_binario["Classificacao"].values

sentimentos_binario = ["Positivo", "Negativo"]


In [54]:
vectorizer_bigram = CountVectorizer(ngram_range = (1, 2))
freq_tweets_bigram = vectorizer_bigram.fit_transform(tweets_binario)

vectorizer_unigram = CountVectorizer(analyzer = "word")
freq_tweets_unigram = vectorizer_unigram.fit_transform(tweets_binario)

In [55]:
gaussian_nb_model_unigram = GaussianNB()
gaussian_nb_model_unigram.fit(freq_tweets_unigram.toarray(), classes_binario)

resultados = cross_val_predict(gaussian_nb_model_unigram, freq_tweets_unigram.toarray(), classes_binario, cv = 10)

array(['Negativo', 'Negativo', 'Positivo', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype='<U8')

In [56]:
metrics.accuracy_score(classes_binario, resultados)

0.9580577793247477

In [58]:
print(metrics.classification_report(classes_binario, resultados, target_names = sentimentos_binario))

              precision    recall  f1-score   support

    Positivo       0.93      0.98      0.95      2446
    Negativo       0.98      0.94      0.96      3300

    accuracy                           0.96      5746
   macro avg       0.96      0.96      0.96      5746
weighted avg       0.96      0.96      0.96      5746



In [59]:
gaussian_nb_model_bigram = GaussianNB()
gaussian_nb_model_bigram.fit(freq_tweets_bigram.toarray(), classes_binario)

resultados = cross_val_predict(gaussian_nb_model_bigram, freq_tweets_bigram.toarray(), classes_binario, cv = 10)

In [60]:
metrics.accuracy_score(classes_binario, resultados)

0.9657152801949183

In [61]:
print(metrics.classification_report(classes_binario, resultados, target_names = sentimentos_binario))

              precision    recall  f1-score   support

    Positivo       0.94      0.98      0.96      2446
    Negativo       0.98      0.96      0.97      3300

    accuracy                           0.97      5746
   macro avg       0.96      0.97      0.97      5746
weighted avg       0.97      0.97      0.97      5746



In [62]:
svm_model_unigram = svm.SVC(kernel='linear')
svm_model_unigram.fit(freq_tweets_unigram, classes_binario)

resultados = cross_val_predict(svm_model_unigram, freq_tweets_unigram, classes_binario, cv = 10)

In [63]:
metrics.accuracy_score(classes_binario, resultados)

0.9570135746606335

In [64]:
print(metrics.classification_report(classes_binario, resultados, target_names = sentimentos_binario))

              precision    recall  f1-score   support

    Positivo       0.92      0.98      0.95      2446
    Negativo       0.98      0.94      0.96      3300

    accuracy                           0.96      5746
   macro avg       0.95      0.96      0.96      5746
weighted avg       0.96      0.96      0.96      5746



In [65]:
svm_model_bigram = svm.SVC(kernel='linear')
svm_model_bigram.fit(freq_tweets_bigram, classes_binario)

resultados = cross_val_predict(svm_model_bigram, freq_tweets_bigram, classes_binario, cv = 10)

In [66]:
metrics.accuracy_score(classes_binario, resultados)

0.9491820396797772

In [67]:
print(metrics.classification_report(classes_binario, resultados, target_names = sentimentos_binario))

              precision    recall  f1-score   support

    Positivo       0.92      0.97      0.94      2446
    Negativo       0.98      0.94      0.95      3300

    accuracy                           0.95      5746
   macro avg       0.95      0.95      0.95      5746
weighted avg       0.95      0.95      0.95      5746

